# **PRÉ-PROCESSAMENTO DOS DADOS**

In [1]:
import numpy as np
import pandas as pd
import re

In [35]:
df = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_raw.csv')
df.head()

,ID,Peso,Altura,IMC,Atendimento,DN,IDADE,Convenio,PULSOS,PA SISTOLICA,...,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO1,MOTIVO2
0,1,5.0,51,19.0,11/05/06,30/03/06,0.12,GS,Normais,NaN,...,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,M,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,2,3.5,50,14.0,25/05/05,19/05/05,0.02,GS,Normais,NaN,...,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,M,6 - Suspeita de cardiopatia,6 - Dispnéia
2,3,0.0,0,NaN,12/06/01,08/05/05,-4.05,SULA,Normais,NaN,...,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,M,2 - Check-up,NaN
3,4,8.1,65,19.0,15/10/09,21/04/09,0.50,NaN,Normais,NaN,...,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,M,5 - Parecer cardiológico,NaN
4,7,40.0,151,18.0,14/01/08,14/08/95,12.89,SAME,Normais,NaN,...,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,M,5 - Parecer cardiológico,NaN


## Retirando a coluna *ID*, *Convenio* e *SEXO*

In [36]:
df_copy = df.copy()
df_copy.drop(columns=['ID', 'Convenio', 'SEXO'], inplace=True)
df_copy.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,MOTIVO1,MOTIVO2
0,5.0,51,19.0,11/05/06,30/03/06,0.12,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,3.5,50,14.0,25/05/05,19/05/05,0.02,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,6 - Suspeita de cardiopatia,6 - Dispnéia
2,0.0,0,NaN,12/06/01,08/05/05,-4.05,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,2 - Check-up,NaN
3,8.1,65,19.0,15/10/09,21/04/09,0.50,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,5 - Parecer cardiológico,NaN
4,40.0,151,18.0,14/01/08,14/08/95,12.89,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,5 - Parecer cardiológico,NaN


## Tratando a coluna *Idade*

In [37]:
def limpar_data(val):
    try:
        # Tenta converter formato dia/mês/ano
        return pd.to_datetime(val, format='%d/%m/%y')
    except:
        try:
            # Se falhar, tenta converter número serial do Excel
            # (Base do Excel é aprox 30/12/1899)
            return pd.to_datetime(float(val), unit='D', origin='1899-12-30')
        except:
            return pd.NaT
    
df_copy['DN'] = df_copy['DN'].apply(limpar_data)
df_copy['Atendimento'] = df_copy['Atendimento'].apply(limpar_data)
df_copy.head()


,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,MOTIVO1,MOTIVO2
0,5.0,51,19.0,2006-05-11,2006-03-30,0.12,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,3.5,50,14.0,2005-05-25,2005-05-19,0.02,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,6 - Suspeita de cardiopatia,6 - Dispnéia
2,0.0,0,NaN,2001-06-12,2005-05-08,-4.05,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,2 - Check-up,NaN
3,8.1,65,19.0,2009-10-15,2009-04-21,0.50,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,5 - Parecer cardiológico,NaN
4,40.0,151,18.0,2008-01-14,1995-08-14,12.89,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,5 - Parecer cardiológico,NaN


In [38]:
df_copy['IDADE'] = (df_copy['Atendimento'] - df_copy['DN']).dt.days / 365

df_copy.loc[df_copy['IDADE'] < 0, 'IDADE'] = np.nan

# Comparação
print("Negativos antes:", (df_copy['IDADE'] < 0).sum())
print("Negativos depois:", (df_copy['IDADE'] < 0).sum())
print("Valores recuperados/corrigidos:", df_copy['IDADE'].notnull().sum())

Negativos antes: 0
Negativos depois: 0
Valores recuperados/corrigidos: 11246


## Tratando a coluna *NORMAL X ANORMAL*

In [40]:
df_copy[(df_copy['NORMAL X ANORMAL'].isna()) & (df_copy['SOPRO'] != 'ausente') & (df_copy['SOPRO'].notna())]

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,MOTIVO1,MOTIVO2


In [39]:
df_copy.dropna(subset=['NORMAL X ANORMAL'], inplace=True)

In [7]:
df_copy.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,MOTIVO1,MOTIVO2
0,5.0,51,19.0,2006-05-11,2006-03-30,0.12,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,3.5,50,14.0,2005-05-25,2005-05-19,0.02,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,6 - Suspeita de cardiopatia,6 - Dispnéia
2,0.0,0,NaN,2001-06-12,2005-05-08,-4.05,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,2 - Check-up,NaN
3,8.1,65,19.0,2009-10-15,2009-04-21,0.50,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,5 - Parecer cardiológico,NaN
4,40.0,151,18.0,2008-01-14,1995-08-14,12.89,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,5 - Parecer cardiológico,NaN


## Tratando coluna *Pulsos*

In [109]:
df_copy['PULSOS'].isna().sum()

np.int64(26)

In [9]:
df_copy[(df_copy['PULSOS'].isna()) & (df_copy['NORMAL X ANORMAL'] == 'Anormal')]

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,MOTIVO1,MOTIVO2
57,32.0,0,NaN,2003-08-06,1993-10-20,10.16,NaN,100.0,60.0,Não Calculado,Anormal,Normal,Sistólico,NaN,Outro,NaN,6 - Suspeita de cardiopatia,6 - Alterações de pulso/perfusão
65,7.1,67,16.0,2004-07-17,2004-02-18,0.43,NaN,NaN,NaN,Não Calculado,Anormal,Normal,sistólico,100,NaN,NaN,6 - Suspeita de cardiopatia,6 - Sopro
84,0.0,0,NaN,2009-04-15,2009-02-27,0.13,NaN,NaN,NaN,Não Calculado,Anormal,Normal,sistólico,200,NaN,NaN,6 - Suspeita de cardiopatia,6 - Sopro
161,5.0,0,NaN,2008-03-12,2008-02-04,0.11,NaN,NaN,NaN,Não Calculado,Anormal,Normal,sistólico,164,Dispneia,NaN,6 - Suspeita de cardiopatia,Outro
167,0.0,0,NaN,2004-01-08,2003-12-04,0.10,NaN,NaN,NaN,Não Calculado,Anormal,Normal,sistólico,152,Dispneia,Cianose,6 - Suspeita de cardiopatia,6 - Sopro
268,0.0,62,0.0,2006-03-07,2005-09-15,0.49,NaN,NaN,NaN,Não Calculado,Anormal,Hiperfonética,ausente,100,NaN,NaN,6 - Suspeita de cardiopatia,6 - Sopro
418,0.0,0,NaN,2008-09-22,2007-06-09,1.34,NaN,NaN,NaN,Não Calculado,Anormal,Única,sistólico,160,NaN,NaN,1 - Cardiopatia já estabelecida,5 - Cirurgia
443,28.0,138,15.0,2007-09-28,1997-09-14,10.41,NaN,130.0,70.0,HAS-2 PAS,Anormal,Normal,sistólico,84,Assintomático,NaN,1 - Cardiopatia já estabelecida,1 - Cardiopatia congenica
450,37.0,151,16.0,2004-10-25,1993-05-08,11.90,NaN,110.0,70.0,Normal,Anormal,Normal,sistólico,80,NaN,NaN,6 - Suspeita de cardiopatia,6 - Sopro
536,0.0,0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,#VALUE!,Anormal,Hiperfonética,Sistólico,NaN,NaN,NaN,NaN,NaN


In [43]:
lista_indices = [
    12562, 12578, 12581, 12590, 12607, 12616, 12634, 12636, 
    12662, 12667, 12688, 12708, 12738, 12748, 12758, 12763, 
    12767, 12782, 12803, 12833, 12839, 12855
]

df_copy.drop(index=lista_indices, inplace=True)

In [10]:
df_copy['PULSOS'].value_counts()

PULSOS
Normais                11509
Amplos                    52
Femorais diminuidos       39
Outro                     36
Diminuídos                18
NORMAIS                    2
AMPLOS                     1
Name: count, dtype: int64

In [110]:
df_copy['PULSOS'] = df_copy['PULSOS'].str.strip().str.capitalize()

df_copy['PULSOS'] = df_copy['PULSOS'].fillna('Normais')

print(df_copy['PULSOS'].value_counts())

PULSOS
Normais                11537
Amplos                    53
Femorais diminuidos       39
Outro                     35
Diminuídos                18
Name: count, dtype: int64


## Tratando coluna *Motivo 1 e 2*

In [44]:
df_copy['MOTIVO1'].isna().sum()

np.int64(265)

In [45]:
df_copy['MOTIVO1'].value_counts()

MOTIVO1
5 - Parecer cardiológico           5178
6 - Suspeita de cardiopatia        4128
1 - Cardiopatia já estabelecida    1130
2 - Check-up                        683
7 - Outro                           299
Name: count, dtype: int64

In [46]:
df_copy['MOTIVO2'].isna().sum()

np.int64(2676)

In [47]:
df_copy['MOTIVO2'].value_counts()

MOTIVO2
5 - Cirurgia                           2846
6 - Sopro                              2284
1 - Cardiopatia congenica              1018
Outro                                   783
5 - Atividade física                    586
6 - Palpitação/taquicardia/arritmia     397
6 - Dor precordial                      369
6 - HAS/dislipidemia/obesidade          226
6 - Dispnéia                            216
6 - Cianose                             137
1 - Cardiopatia adquirida                90
6 - Cardiopatia na familia               32
5 - Uso de cisaprida                      9
6 - Cansaço                               6
6 - Alterações de pulso/perfusão          5
6 - Cianose e dispnéia                    3
Name: count, dtype: int64

In [48]:
def limpar_motivo(valor):
    if pd.isna(valor):
        return np.nan
    
    texto = str(valor)

    texto_limpo = re.sub(r'^\d+\s*-\s*', '', texto)

    return texto_limpo.strip().lower()

df_copy['MOTIVO1'] = df_copy['MOTIVO1'].apply(limpar_motivo)
df_copy['MOTIVO2'] = df_copy['MOTIVO2'].apply(limpar_motivo)

df_copy['MOTIVO'] = df_copy['MOTIVO2'].fillna(df_copy['MOTIVO1'])

df_copy.drop(columns=['MOTIVO1', 'MOTIVO2'], inplace=True)
df_copy.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,MOTIVO
0,5.0,51,19.0,2006-05-11,2006-03-30,0.115068,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,palpitação/taquicardia/arritmia
1,3.5,50,14.0,2005-05-25,2005-05-19,0.016438,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,dispnéia
2,0.0,0,NaN,2001-06-12,2005-05-08,NaN,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,check-up
3,8.1,65,19.0,2009-10-15,2009-04-21,0.484932,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,parecer cardiológico
4,40.0,151,18.0,2008-01-14,1995-08-14,12.427397,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,parecer cardiológico


## Tratando colunas *HDA 1 e 2*

In [49]:
df_copy['HDA 1'].isna().sum()

np.int64(3119)

In [50]:
df_copy['HDA 1'].value_counts()

HDA 1
Assintomático      6532
Dispneia            539
Dor precordial      507
Palpitacao          362
Cianose             258
Desmaio/tontura     155
Outro               124
Ganho de peso        87
Name: count, dtype: int64

In [51]:
df_copy['HDA2'].isna().sum()

np.int64(11281)

In [52]:
df_copy['HDA2'].value_counts()

HDA2
Palpitacao         83
Dispneia           82
Dor precordial     65
Cianose            62
Desmaio/tontura    49
Outro              41
Ganho de peso      19
Assintomático       1
Name: count, dtype: int64

In [53]:
def unir_sintomas(row):
    sintomas = set()

    if pd.notna(row['HDA 1']):
        sintoma1 = str(row['HDA 1']).strip().lower()
        if sintoma1 not in ['', 'nan']:
            sintomas.add(sintoma1)

    if pd.notna(row['HDA2']):
        sintoma2 = str(row['HDA2']).strip().lower()
        if sintoma2 not in ['', 'nan']:
            sintomas.add(sintoma2)

    return ",".join(sintomas) 

df_copy['HDA'] = df_copy.apply(unir_sintomas, axis=1)

df_copy.drop(columns=['HDA 1', 'HDA2'], inplace=True)

print("Coluna unida:")
print(df_copy['HDA'].head())

Coluna unida:
0        palpitacao
1          dispneia
2     assintomático
3     assintomático
4    dor precordial
Name: HDA, dtype: object


In [54]:
df_copy.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,MOTIVO,HDA
0,5.0,51,19.0,2006-05-11,2006-03-30,0.115068,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,palpitação/taquicardia/arritmia,palpitacao
1,3.5,50,14.0,2005-05-25,2005-05-19,0.016438,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,dispnéia,dispneia
2,0.0,0,NaN,2001-06-12,2005-05-08,NaN,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,check-up,assintomático
3,8.1,65,19.0,2009-10-15,2009-04-21,0.484932,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,parecer cardiológico,assintomático
4,40.0,151,18.0,2008-01-14,1995-08-14,12.427397,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,parecer cardiológico,dor precordial


## Tratando coluna *PPA*

In [78]:
df_copy['PPA'].isna().sum()

np.int64(217)

In [79]:
df_copy['PPA'].value_counts()

PPA
Não Calculado          8681
Normal                 1790
#VALUE!                 706
HAS-2 PAS                73
Pre-Hipertensão PAD      73
Pre-Hipertensão PAS      48
HAS-1 PAS                48
HAS-1 PAD                26
HAS-2 PAD                21
Name: count, dtype: int64

In [85]:
df_copy['PA DIASTOLICA'].isna().sum()

np.int64(6562)

## Traatando a coluna *B2*

In [55]:
df_copy['B2'].isna().sum()

np.int64(9)

In [19]:
df_copy['B2'].value_counts()

B2
Normal           11077
Hiperfonética      295
Desdob fixo        149
Única               77
Outro               76
Name: count, dtype: int64

In [56]:
df_copy['B2'] = df_copy['B2'].fillna('Normal')

## Tratando coluna *SOPRO*

In [25]:
df_copy['SOPRO'].isna().sum()

np.int64(0)

In [61]:
df_copy['SOPRO'].value_counts()

SOPRO
ausente                   7260
Sistólico                 3640
sistólico                  723
contínuo                    28
Contínuo                    19
diastólico                   9
Sistolico e diastólico       3
Name: count, dtype: int64

In [57]:
df_copy[df_copy['SOPRO'].isna()]

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,MOTIVO,HDA
1951,0.0,0,NaN,2009-04-08,2003-08-21,5.635616,Outro,NaN,NaN,Não Calculado,Normal,Normal,NaN,NaN,cirurgia,


In [58]:
df_copy.dropna(subset=['SOPRO'], inplace=True)

In [63]:
print(len(df_copy))

11691


## Retirar linhas ou colunas com mais de 50% de dados faltantes

In [60]:
(df_copy.isna().mean() * 100).sort_values(ascending=False)

PA DIASTOLICA       56.163328
PA SISTOLICA        56.077726
IMC                 33.145009
IDADE                7.130628
DN                   6.043486
FC                   5.957884
Peso                 2.653655
MOTIVO               2.268447
PPA                  1.857559
Atendimento          1.806198
PULSOS               0.222565
Altura               0.000000
B2                   0.000000
NORMAL X ANORMAL     0.000000
SOPRO                0.000000
HDA                  0.000000
dtype: float64

In [61]:
df_copy.drop(columns=['PA DIASTOLICA', 'PA SISTOLICA'], inplace=True)

In [62]:
(df_copy.isna().mean() * 100).sort_values(ascending=False)

IMC                 33.145009
IDADE                7.130628
DN                   6.043486
FC                   5.957884
Peso                 2.653655
MOTIVO               2.268447
PPA                  1.857559
Atendimento          1.806198
PULSOS               0.222565
Altura               0.000000
B2                   0.000000
NORMAL X ANORMAL     0.000000
SOPRO                0.000000
HDA                  0.000000
dtype: float64

In [63]:
# 1. Calcula a porcentagem de nulos para cada linha
percentual_nulos_linha = df_copy.isna().mean(axis=1)

# 2. Filtra apenas as linhas com mais de 50% (0.5) de nulos
linhas_vazias = df_copy[percentual_nulos_linha > 0.5]

# Mostra quantas linhas foram encontradas
print(f"Foram encontradas {len(linhas_vazias)} linhas com mais de 50% de dados faltantes.")

# Visualiza essas linhas (mostrando apenas as primeiras 5 para não poluir a tela)
linhas_vazias.head()

Foram encontradas 0 linhas com mais de 50% de dados faltantes.


,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,MOTIVO,HDA


In [64]:
# 1. Identifica os índices das linhas com mais de 50% (0.5) de nulos
linhas_para_remover = df_copy[df_copy.isna().mean(axis=1) > 0.5].index

# 2. Remove as linhas usando .drop()
df_copy.drop(linhas_para_remover, inplace=True)

# 3. Feedback para você saber quantas foram apagadas
print(f"Foram removidas {len(linhas_para_remover)} linhas.")

Foram removidas 0 linhas.


In [65]:
df_copy.drop(columns=['DN', 'Atendimento'], inplace=True)

In [66]:
df_copy.head()

,Peso,Altura,IMC,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,MOTIVO,HDA
0,5.0,51,19.0,0.115068,Normais,Não Calculado,Anormal,Normal,Sistólico,112,palpitação/taquicardia/arritmia,palpitacao
1,3.5,50,14.0,0.016438,Normais,Não Calculado,Anormal,Normal,ausente,128,dispnéia,dispneia
2,0.0,0,NaN,NaN,Normais,Não Calculado,Anormal,Normal,Sistólico,88,check-up,assintomático
3,8.1,65,19.0,0.484932,Normais,Não Calculado,Anormal,Normal,ausente,92,parecer cardiológico,assintomático
4,40.0,151,18.0,12.427397,Normais,Não Calculado,Anormal,Normal,ausente,96,parecer cardiológico,dor precordial


In [112]:
df_copy.to_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_cleaned.csv', index=False)

## Tratando as colunas numéricas

In [82]:
cols_biologicas = ['Peso', 'Altura', 'IDADE']

for col in cols_biologicas:
    if col in df_copy.columns:
        qtd_erros = df_copy[df_copy[col] <= 0].shape[0]
        if qtd_erros > 0:
            print(f"Coluna '{col}': {qtd_erros} valores inválidos (<=0) transformados em NaN.")
            
            df_copy.loc[df_copy[col] <= 0, col] = np.nan


mask_imc = df_copy['Peso'].notnull() & df_copy['Altura'].notnull() & df_copy['IMC'].isnull()
df_copy.loc[mask_imc, 'IMC'] = df_copy.loc[mask_imc, 'Peso'] / ((df_copy.loc[mask_imc, 'Altura']/100) ** 2)

for col in cols_biologicas + ['IMC']:
    if col in df_copy.columns:
        mediana = df_copy[col].median()
        df_copy[col] = df_copy[col].fillna(mediana)

print("\nVerificação Final de Nulos:")
print(df_copy[cols_biologicas].isnull().sum())


Verificação Final de Nulos:
Peso      0
Altura    0
IDADE     0
dtype: int64


In [86]:
df_copy.head()

,Peso,Altura,IMC,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,MOTIVO,HDA
0,5.0,51.0,19.0,0.115068,Normais,Não Calculado,Anormal,Normal,Sistólico,112,palpitação/taquicardia/arritmia,palpitacao
1,3.5,50.0,14.0,0.016438,Normais,Não Calculado,Anormal,Normal,ausente,128,dispnéia,dispneia
2,16.0,99.0,17.0,3.823288,Normais,Não Calculado,Anormal,Normal,Sistólico,88,check-up,assintomático
3,8.1,65.0,19.0,0.484932,Normais,Não Calculado,Anormal,Normal,ausente,92,parecer cardiológico,assintomático
4,40.0,151.0,18.0,12.427397,Normais,Não Calculado,Anormal,Normal,ausente,96,parecer cardiológico,dor precordial


In [111]:
df_copy.isna().sum()

Peso                  0
Altura                0
IMC                   0
IDADE                 0
PULSOS                0
PPA                 217
NORMAL X ANORMAL      0
B2                    0
SOPRO                 0
FC                    0
MOTIVO              265
HDA                   0
dtype: int64

## Tratando *FC*

In [88]:
def clean_fc(val):
    if pd.isna(val):
        return np.nan
    
    val_str = str(val).strip()
    
    if '-' in val_str:
        try:
            parts = val_str.split('-')
            return np.mean([float(p) for p in parts])
        except:
            return np.nan
    if ' a ' in val_str:
        try:
            parts = val_str.split(' a ')
            return np.mean([float(p) for p in parts])
        except:
            return np.nan

    try:
        num = float(val_str)
    except ValueError:
        return np.nan

    if num > 300: 
        if num < 3000: # Provável erro de x10 (ex: 1120 -> 112)
             num = num / 10
        elif num < 30000: # Provável erro de x100 (ex: 9288 -> 92.88)
             num = num / 100
        else:
             return np.nan # Dado irrecuperável

    if num < 30: 
        return np.nan

    return round(num)

df_copy['FC'] = df_copy['FC'].apply(clean_fc)

In [98]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11682 entries, 0 to 12872
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Peso              11682 non-null  float64
 1   Altura            11682 non-null  float64
 2   IMC               11682 non-null  float64
 3   IDADE             11682 non-null  float64
 4   PULSOS            11656 non-null  object 
 5   PPA               11465 non-null  object 
 6   NORMAL X ANORMAL  11682 non-null  object 
 7   B2                11682 non-null  object 
 8   SOPRO             11682 non-null  object 
 9   FC                10980 non-null  float64
 10  MOTIVO            11417 non-null  object 
 11  HDA               11682 non-null  object 
dtypes: float64(5), object(7)
memory usage: 1.2+ MB


In [101]:
df_copy['Age_Int'] = df_copy['IDADE'].apply(lambda x: int(x) if pd.notnull(x) else x)

df_copy['FC'] = df_copy['FC'].fillna(df_copy.groupby('Age_Int')['FC'].transform('median'))

df_copy['FC'] = df_copy['FC'].fillna(df_copy['FC'].median())

df_copy.drop(columns=['Age_Int'], inplace=True)

In [102]:
df_copy['FC'].isna().sum()

np.int64(0)

In [103]:
df_copy['FC'].describe()

count    11682.000000
mean        95.269432
std         17.795335
min         43.000000
25%         80.000000
50%         92.000000
75%        100.000000
max        300.000000
Name: FC, dtype: float64